In [1]:
from torch import nn
import numpy as np
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
from sklearn.manifold import TSNE
import seaborn as sns
import torch
import torch.nn.functional as F
import warnings
warnings.filterwarnings('ignore')
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import clear_output
import plotly.offline as pyo
import plotly.io as pio
pio.renderers.default = 'iframe'
import wandb
from sklearn.metrics import classification_report, accuracy_score
from ConvTranModel import ConvTranWrapper
from torchmetrics import Accuracy

# Generate Time Series

In [2]:
np.random.seed(42)
N = 20
L = 500
n_obs = 5000
beta = 3
y = np.random.randint(0, 2, size = n_obs)
data = []
for i in tqdm(range(n_obs)):
    data.append(np.random.normal(0, 1 + y[i]*beta, size = (N,L)))
data = torch.tensor(data, dtype = torch.float)
y = torch.tensor(y)

100%|██████████| 5000/5000 [00:02<00:00, 2224.51it/s]


In [3]:
data.size(), data.type()

(torch.Size([5000, 20, 500]), 'torch.FloatTensor')

In [4]:
class TSC_Data(Dataset):
    def __init__(self, data, y):
        super().__init__()
        self.data = data
        self.y = y.long()
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.data[idx], self.y[idx]

In [5]:
train_size = 3000
val_size = 1000
train_set = TSC_Data(data[:train_size], y[:train_size])
val_set = TSC_Data(data[train_size:train_size+val_size], y[train_size:train_size+val_size])
test_set = TSC_Data(data[train_size+val_size:], y[train_size+val_size:])

In [6]:
train_loader = DataLoader(train_set, batch_size = 16, shuffle = True)
val_loader = DataLoader(train_set, batch_size = 16, shuffle = True)
test_loader = DataLoader(test_set, batch_size = 16, shuffle = True)

In [7]:
wandb.login()

wandb: Currently logged in as: gerasimov-serf (zimran-c01-hack). Use `wandb login --relogin` to force relogin


True

In [8]:
project = 'TSC'
name = 'ConvTran on toy dataset'

In [8]:
tsc_model = ConvTranWrapper(project, name, data.shape, emb_size = 64, dim_ff = 64, num_heads = 2, 
                            fix_pos_encode = 'tAPE', rel_pos_encode = 'eRPE')

In [16]:
# tsc_model.fit(train_loader, val_loader)

In [14]:
accuracy, _ = tsc_model.test(test_loader)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       511
           1       1.00      1.00      1.00       489

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000



In [15]:
wandb.finish()

# Lightning module

In [9]:
import pytorch_lightning as pl
from lightning.pytorch.loggers import WandbLogger
from ConvTranModel import ConvTran

In [11]:
class LitConvTran(pl.LightningModule):
    def __init__(self, project, name, data_shape, emb_size = 64, dim_ff = 64, num_heads = 2, 
                            fix_pos_encode = 'tAPE', rel_pos_encode = 'eRPE', lr = 0.01, num_classes = 2, dropout = 0):
        super().__init__()
        
        # log hyperparameters
        self.save_hyperparameters()
        self.project, self.name = project, name
        self.config = {
            'Data_shape' : data_shape, 
            'emb_size' : emb_size, 
            'dim_ff' : dim_ff, 
            'Fix_pos_encode' : fix_pos_encode,
            'Rel_pos_encode' : rel_pos_encode,
            'num_classes' : num_classes,
            'num_heads': num_heads,
            'dropout' : dropout, 
            'lr' : lr,
        }
        self.convtran = ConvTran(self.config, num_classes)
        self.accuracy = Accuracy(task='binary')
        
    def forward(self, x):
        return self.convtran(x)
        
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        
        # training metrics
        preds = torch.argmax(logits, dim=1)
        acc = self.accuracy(preds, y)
        self.log('train_loss', loss, on_step=True, on_epoch=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, logger=True)
        return loss
        
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)

        # validation metrics
        preds = torch.argmax(logits, dim=1)
        acc = self.accuracy(preds, y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        return loss
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        
        # validation metrics
        preds = torch.argmax(logits, dim=1)
        acc = self.accuracy(preds, y)
        self.log('test_loss', loss, prog_bar=True)
        self.log('test_acc', acc, prog_bar=True)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.config['lr'])
        return optimizer

In [12]:
model = LitConvTran(project, name, data.shape, emb_size = 64, dim_ff = 64, num_heads = 2, 
                            fix_pos_encode = 'tAPE', rel_pos_encode = 'eRPE', lr = 0.01)

# Initialize wandb logger
wandb_logger = WandbLogger(project=project, name = name+'Lite',job_type='train')

In [13]:
# Initialize Callbacks
early_stop_callback = pl.callbacks.EarlyStopping(monitor="val_loss")
checkpoint_callback = pl.callbacks.ModelCheckpoint()

In [14]:
# Initialize a trainer
trainer = pl.Trainer(max_epochs=3,
                     logger=wandb_logger,
                     callbacks=[early_stop_callback,
                                checkpoint_callback],
                     )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [15]:
# Train the model
trainer.fit(model = model, train_dataloaders = train_loader, val_dataloaders=val_loader)

You are using a CUDA device ('NVIDIA GeForce RTX 3050 Ti Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type           | Params
--------------------------------------------
0 | convtran | ConvTran       | 353 K 
1 | accuracy | BinaryAccuracy | 0     
--------------------------------------------
353 K     Trainable params
0         Non-trainable params
353 K     Total params
1.415     Total estimated model params size (MB)


Epoch 0: 100%|██████████| 188/188 [00:17<00:00, 11.04it/s, v_num=zk7b]     
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 188/188 [00:16<00:00, 11.62it/s, v_num=zk7b, val_loss=3.02e-5, val_acc=1.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 188/188 [00:16<00:00, 11.60it/s, v_num=zk7b, val_loss=1.39e-5, val_acc=1.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 188/188 [00:21<00:00,  8.85it/s, v_num=zk7b, val_loss=8.18e-6, val_acc=1.000]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 188/188 [00:21<00:00,  8.81it/s, v_num=zk7b, val_loss=8.18e-6, val_acc=1.000]


In [16]:
# Evaluate the model on the held-out test set 
trainer.test(dataloaders=test_loader)

Restoring states from the checkpoint path at .\TSC\yfhyzk7b\checkpoints\epoch=2-step=564.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at .\TSC\yfhyzk7b\checkpoints\epoch=2-step=564.ckpt


Testing DataLoader 0: 100%|██████████| 63/63 [00:01<00:00, 40.60it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc                    1.0
        test_loss          8.192387213057373e-06
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 8.192387213057373e-06, 'test_acc': 1.0}]

In [17]:
# Close wandb run
wandb.finish()

epoch,▁▁▁▁▁▃▃▃▃▃▃▆▆▆▆▆▆█
test_acc,▁
test_loss,▁
train_acc_epoch,▁██
train_acc_step,▁▁▁▁▁▁▁▁▁▁▁
train_loss_epoch,█▁▁
train_loss_step,█▇▄▃▂▂▁▁▁▁▁
trainer/global_step,▁▂▂▃▃▃▄▄▅▅▅▆▆▇████
val_acc,▁▁▁
val_loss,█▃▁
epoch,3
